In [1]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

<img title="a title" alt="Alt text" src="https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png">

In [3]:
include_points = [23,24,25,26,27,28]
def create_df():
    columns=['Form']

    for pt in include_points:
        landmark = str(mp_pose.PoseLandmark(pt))
        columns.append(landmark[13:]+ '_x')
        columns.append(landmark[13:]+ '_y')

    return pd.DataFrame(columns=columns)

In [4]:
def calulate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle

    return angle

In [5]:
def calculate_disance(a,b):
    a = np.array(a)
    b = np.array(b)

    return np.linalg.norm(a-b)

In [6]:
def record_data(name, label):
    df = create_df()

    cap = cv2.VideoCapture(name)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            if ret==False:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)
            left_shoulder = [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            right_shoulder = [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
            shoulder_distance = calculate_disance(left_shoulder, right_shoulder)
            row=[label]
            for pt in include_points:
                row.append(results.pose_landmarks.landmark[pt].x/shoulder_distance)
                row.append(results.pose_landmarks.landmark[pt].y/shoulder_distance)
            df.loc[len(df)] = row

            #Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            resize = cv2.resize(image, (405, 720))

            # Render detections
            mp_drawing.draw_landmarks(resize, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            cv2.imshow('MediaPipe Pose', resize)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    return df

In [7]:
for root, dirs, files in os.walk("vids/correct"):
    for index, file in enumerate(files):
        if file.endswith(".mp4"):
            df = record_data(os.path.join(root, file), "proper")
            df.to_csv('out/csv/correct/'+str(index+1)+'.csv', index=False)

In [8]:
for root, dirs, files in os.walk("vids/incorrect"):
    for index, file in enumerate(files):
        if file.endswith(".mp4"):
            df = record_data(os.path.join(root, file), "icomplete")
            df.to_csv('out/csv/incorrect/'+str(index+1)+'.csv', index=False)

<img >